SyntaxError: invalid syntax (2125240012.py, line 1)

In [5]:
import pyspark
import timeit
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, mean, count

In [2]:
emails = "emails.csv"

In [3]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("emails_session").getOrCreate()
spark

23/04/25 20:57:52 WARN Utils: Your hostname, Merons-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.106 instead (on interface en0)
23/04/25 20:57:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/25 20:57:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [17]:
# writing the basic funtions


In [18]:
# Read the CSV file

def read_csv():
    df = spark.read.option('header', 'true').csv(emails, inferSchema=True)
    return df

In [19]:
# Filtering and selection

def filter_emails(df):
    filtered_df = df.filter(col('subject').like('%Enron%'))
    return filtered_df

In [20]:
# GroupBy and aggregation

def groupby_emails(df):
    grouped_df = df.groupBy('from').agg(sum('size'), mean('size'), count('size'))
    return grouped_df

In [ ]:
# Data transformation

def transform_emails(df):
    transformed_df = df.withColumn('size_kb', col('size') / 1024)
    return transformed_df

In [15]:
methods = [read_csv, filter_emails, groupby_emails, transform_emails]
method_names = ['Read CSV', 'Filter & Select', 'GroupBy & Aggregation', 'Data Transformation']


In [16]:
for method, method_name in zip(methods, method_names):
    timings = []
    
    for _ in range(5):
        start_time = timeit.default_timer()
        df = read_csv()
        method(df)
        end_time = timeit.default_timer()
        timings.append(end_time - start_time)
    avg_time = sum(timings) / len(timings)
    

TypeError: read_csv() takes 0 positional arguments but 1 was given